# PBupsModelmusc with Algorithmic Differentiation in Julia
3 additional parameters were added to Bing’s accumulator model.  
To analyze the strong bias induced by unilateral FOF inactivation


In [1]:
# import packages..
import ForwardDiff
using ForwardDiff
using DiffBase
using PyPlot
import Base.convert
import Optim
using Optim

# === Upgrading from ForwardDiff v0.1 to v0.2
# instead of ForwardDiff.GradientNumber and ForwardDiff.HessianNumber, 
# we will use ForwardDiff.Dual

convert(::Type{Float64}, x::ForwardDiff.Dual) = Float64(x.value)
function convert(::Array{Float64}, x::Array{ForwardDiff.Dual}) 
    y = zeros(size(x)); 
    for i in 1:prod(size(x)) 
        y[i] = convert(Float64, x[i]) 
    end
    return y
end

immutable NumericPair{X,Y} <: Number
  x::X
  y::Y
end
Base.isless(a::NumericPair, b::NumericPair) = (a.x<b.x) || (a.x==b.x && a.y<b.y)


    write(Base.IO, ForwardDiff.Partials) at /Users/msyoon/.julia/v0.4/ForwardDiff/src/partials.jl:57
is ambiguous with: 
    write(Base.Base64.Base64EncodePipe, AbstractArray{UInt8, 1}) at base64.jl:89.
To fix, define 
    write(Base.Base64.Base64EncodePipe, ForwardDiff.Partials{N<:Any, UInt8})
before the new definition.


isless (generic function with 40 methods)

To evaluate how well a particular set of parameter values $\theta$ fits the behavioral data, we compute the probability of oberving the data given the model.

For each trial $i$, we will compute the likelihood of seeing the data under the model assuming that trials are independent. 

$P(D|\theta) = \prod_{i}P(d_i|t_{i,R},t_{i,L},\theta)$

$t_{i,R},t_{i,L}$ : the right and left click times on trial $i$

$d_i$ : the subject's decision on trial $i$

The best-fit parameter values are the parameters $\theta$ that maximize the likelihood (Maximum likelihood values)

To help maximize the likelihood(or log likelihood), we will compute the derivative $\partial P(d_i|t_{i,R},t_{i,L},\theta) / \partial\theta$ for each of the parameters in the set $\theta$.

After we get these gradients of 9 model parameters, we will apply them for optimization.

## Import data 

In [2]:
using MAT
ratdata = matread("chrono_fof_rawdata.mat")
# ratdata = matread("testdata.mat")
# ratdata = matread("chrono_B069_rawdata.mat")

Dict{ASCIIString,Any} with 2 entries:
  "rawdata"      => Dict{ASCIIString,Any}("is_probe"=>1x3836 Array{Any,2}:…
  "orig_rawdata" => Dict{ASCIIString,Any}("is_probe"=>1x3836 Array{Any,2}:…

In [3]:
function trialdata(rawdata, trial::Int)
    if rawdata["pokedR"][trial] > 0
        rat_choice = 1;  # "R"
    else
        rat_choice = -1; # "L"
    end;
    
    if typeof(rawdata["rightbups"][trial]) <: Array
        rvec = vec(rawdata["rightbups"][trial])::Array{Float64,1};
    else
        rvec = Float64[]
    end
    if typeof(rawdata["leftbups"][trial]) <: Array
        lvec = vec(rawdata["leftbups"][trial])::Array{Float64,1};
    else
        lvec = Float64[]
    end
    
    return rvec, lvec, 
    rawdata["T"][trial]::Float64, rat_choice
end



trialdata (generic function with 1 method)

In [4]:
@time RightClickTimes, LeftClickTimes, maxT, rat_choice = trialdata(ratdata["rawdata"], 1)

  0.150032 seconds (29.95 k allocations: 1.293 MB)


([0.0,0.012634999999999999],[0.0,0.028775000000000002,0.046955,0.07099,0.10994,0.12017,0.21356,0.21445,0.230355,0.26368,0.266795],0.2796896469572883,-1)

## bin_centers = make_bins(B, dx, binN)

In [5]:
"""
function bin_centers = make_bins(B, dx, binN)

Makes a series of points that will indicate bin centers. The first and
last points will indicate sticky bins. No "bin edges" are made-- the edge
between two bins is always implicity at the halfway point between their
corresponding centers. The center bin is always at x=0; bin spacing
(except for last and first bins) is always dx; and the position
of the first and last bins is chosen so that |B| lies exactly at the
midpoint between 1st (sticky) and 2nd (first real) bins, as well as
exactly at the midpoint between last but one (last real) and last
(sticky) bins.

Playing nice with ForwardDiff means that the *number* of bins must be predetermined.
So this function will not actually set the number of bins; what it'll do is determine their
locations. To accomplish this separation, the function uses as a third parameter binN,
which should be equal to the number of bins with bin centers > 0, as follows: 
   binN = ceil(B/dx)
and then the total number of bins will be 2*binN+1, with the center one always corresponding
to position zero. Use non-differentiable types for B and dx for this to work.
"""

function make_bins{T}(bins::Vector{T}, B, dx::T, binN)
    cnt = 1
    for i=-binN:binN
        bins[cnt] = i*dx
        cnt = cnt+1
    end
    
    if binN*dx == B
        bins[end] = B + dx
        bins[1] = -B - dx
    else
        bins[end] = 2*B - (binN-1)*dx
        bins[1] = -2*B + (binN-1)*dx
    end
end;

In [6]:
binN = ceil(4.1/0.25)
bins = zeros(typeof(binN), Int(binN*2+1))
@time make_bins(bins,4.1,0.25,binN)
bins

  0.069064 seconds (63.98 k allocations: 2.535 MB)


35-element Array{Float64,1}:
 -4.2 
 -4.0 
 -3.75
 -3.5 
 -3.25
 -3.0 
 -2.75
 -2.5 
 -2.25
 -2.0 
 -1.75
 -1.5 
 -1.25
  ⋮   
  1.5 
  1.75
  2.0 
  2.25
  2.5 
  2.75
  3.0 
  3.25
  3.5 
  3.75
  4.0 
  4.2 

In [7]:
B = 4.1
dx_test = 0.25
binN = ceil(B/dx_test)
bins = zeros(typeof(binN), Int(binN*2+1))
@profile make_bins(bins,B,dx_test,binN)
Profile.print()
Profile.clear_malloc_data() 
bin_centers = bins
bins

1 task.jl; anonymous; line: 447
 1 ...IJulia/src/eventloop.jl; eventloop; line: 8
  1 ...src/execute_request.jl; execute_request; line: 157
   1 loading.jl; include_string; line: 282
    1 ...ia/lib/julia/sys.dylib; typeinf_ext; (unknown line)
     1 ...a/lib/julia/sys.dylib; typeinf; (unknown line)
      1 ...a/lib/julia/sys.dylib; typeinf_uncached; (unknown line)
       1 ...a/lib/julia/sys.dylib; abstract_interpret; (unknown line)
        1 .../lib/julia/sys.dylib; abstract_eval; (unknown line)
         1 .../lib/julia/sys.dylib; abstract_eval_call; (unknown line)
          1 ...lib/julia/sys.dylib; abstract_eval; (unknown line)
           1 ...lib/julia/sys.dylib; abstract_eval_constant; (unknown line)


35-element Array{Float64,1}:
 -4.2 
 -4.0 
 -3.75
 -3.5 
 -3.25
 -3.0 
 -2.75
 -2.5 
 -2.25
 -2.0 
 -1.75
 -1.5 
 -1.25
  ⋮   
  1.5 
  1.75
  2.0 
  2.25
  2.5 
  2.75
  3.0 
  3.25
  3.5 
  3.75
  4.0 
  4.2 

## Global Variables

In [8]:
# Global variables 
const epsilon = 10.0^(-10);
const dx = 0.25;
const dt = 0.02;
const total_rate = 40;

## Parameters

a : decision variable, memory accumulator

$$ da =
  \begin{cases}
    0       & \quad \text{if, } |a| \geq B \\
    \sigma_adW + (\delta_{t,t_R} \cdot \eta C(t) - \delta_{t,t_L} \cdot \eta C(t))dt + \lambda adt  & \quad \text{otherwise, }\\
  \end{cases}
$$



The impact of each click (C) is affected by sensory adaptation that depends on clicks from both right and left sides:

$$ 
\frac{\mathrm d C}{\mathrm d t} = \frac{1-C}{\tau_\phi} + (1-\phi)C(\delta_{t,t_R}+\delta_{t,t_L}) 
$$


sigma2_a ($\sigma_a^2$) : a diffusion constant, parameterizing noise in a.

sigma2_s_R ($\sigma_{s,R}^2$) : parameterizing noise when adding evidence from a right pulse. (incoming sensory evidence)

sigma2_s_L ($\sigma_{s,L}^2$) : parameterizing noise when adding evidence from a left pulse. (incoming sensory evidence)

sigma2_i ($\sigma_i^2$) : initial condition for the dynamical equation at $t=0$

lam ($\lambda$) : consistent drift in the memory a ($\lambda<0$ : leaky or forgetful case, $\lambda>0$ : unstable or impulsive case)

B : decision bound

bias : bias parameter determines the position of the threshold in a (which a Rightward decision is made) accumulation shift

phi ($\phi$) : parameterize sensory adaptation (by defining the dynamics of C ($\phi>1$ : Facilitation, $\phi<1$ : Depression, $\phi=1$ : absense of sensory adaptation)

tau_phi ($\tau_\phi$) : recovery time constant for C. C recovers toward an unadapted value of 1 with time constant $\tau_\phi$

lapse_R($\kappa_R$) : The lapse rate parameterizes the probability of making a random response as right choice. (L->R)

lapse_L($\kappa_L$) : The lapse rate parameterizes the probability of making a random response as left choice. (R->L)

biased_input : unbalanced input gain (sensory neglect)


In [9]:
sigma_a = 1; sigma_s = 0.1; sigma_i = 0.2; 
sigma_a_sbin = sigma_a  # remember we need this copy for Fmatrix
lam = -0.0005; B = 4.1; bias = 0.1; 
phi = 0.3; tau_phi = 0.1; lapse = 0.05*2;
biased_sigma2_s = 0.1; biased_input = 0.1; biased_lapse = 0.1;
params = [sigma_a, sigma_s, sigma_i, lam, B, bias, phi, tau_phi, lapse, biased_sigma2_s, biased_input, biased_lapse]   

sigma = params[1];
lam   = params[2];
c     = params[3];

## F = Fmatrix([sigma, lambda, c], bin_centers)

In [10]:
"""
function F = Fmatrix([sigma, lambda, c], bin_centers)

Uses globals
    dt
    dx
    epsilon       (=10.0^-10)

Returns a square Markov matrix of transition probabilities. 
Plays nice with ForwardDiff-- that is why bin_centers is a global vector (so that the rem
operations that go into defining the bins, which ForwardDiff doesn't know how to deal with,
stay outside of this differentiable function)

sigma  should be in (accumulator units) per (second^(1/2))
lambda should be in s^-1
c      should be in accumulator units per second
bin_centers should be a vector of the centers of all the bins. Edges will be at midpoints
       between the centers, and the first and last bin will be sticky.

dx is not used inside Fmatrix, because bin_centers specifies all we need to know.
dt *is* used inside Fmatrix, to convert sigma, lambda, and c into timestep units
"""
function Fmatrix{T}(F::AbstractArray{T,2},params::Vector, bin_centers)
    sigma2 = params[1];
    lam   = params[2];
    c     = params[3];

    sigma2_sbin = convert(Float64, sigma2)

    if dx > epsilon && sigma2_sbin > 0 
        n_sbins = max(70, ceil(10*sqrt(sigma2_sbin)/dx))
    else
        n_sbins = 70
    end

    F[1,1] = 1;
    F[end,end] = 1;

    if sigma2_sbin <= 0
        sbin_length = 1;
        base_sbins = 0;
        ps = 0;
    else
        swidth = 5*sqrt(sigma2_sbin)
        sbinsize = swidth/n_sbins;#sbins[2] - sbins[1]
        base_sbins    = collect(-swidth:sbinsize:swidth)

        ps       = exp(-base_sbins.^2/(2*sigma2))
        ps       = ps/sum(ps);

        sbin_length = length(base_sbins)
    end
    
    binN = length(bin_centers)

    mu = 0.
    for j in 2:binN-1
        if abs(lam) < epsilon 
            mu = bin_centers[j] + c*dt#(exp(lam*dt))
        else
            mu = (bin_centers[j] + c/lam)*exp(lam*dt) - c/lam
        end

        for k in 1:sbin_length
            sbin = mu + base_sbins[k]#(k-1)*sbinsize + mu - swidth

            if sbin <= bin_centers[1] #(bin_centers[1] + bin_centers[2])/2
                F[1,j] = F[1,j] + ps[k]
            elseif bin_centers[end] <= sbin#(bin_centers[end]+bin_centers[end-1])/2 <= sbins[k]
                F[end,j] = F[end,j] + ps[k]
            else # more condition
                if (sbin > bin_centers[1] && sbin < bin_centers[2])
                    lp = 1; hp = 2;
                elseif (sbin > bin_centers[end-1] && sbin < bin_centers[end])
                    lp = binN-1; hp = binN;
                else
                    lp = floor(Int,((sbin-bin_centers[2])/dx)) + 2#find(bin_centers .<= sbins[k])[end]
                    hp = ceil(Int,((sbin-bin_centers[2])/dx)) + 2#lp+1#Int(ceil((sbins[k]-bin_centers[2])/dx) + 1);
                end

                # if lp < 1
                #     lp = 1;
                # end
                # if hp > binN-1
                #     hp = binN-1;
                # end

                if lp == hp
                    F[lp,j] = F[lp,j] + ps[k]
                else
                    F[hp,j] = F[hp,j] + ps[k]*(sbin - bin_centers[lp])/(bin_centers[hp] - bin_centers[lp])
                    F[lp,j] = F[lp,j] + ps[k]*(bin_centers[hp] - sbin)/(bin_centers[hp] - bin_centers[lp])
                end
            end
        end
    end
    # F[:,1] = 0; F[:,end] = 0; F[1,1] = 1; F[end,end] = 1;
end


Fmatrix (generic function with 1 method)

In [11]:
F = zeros(typeof(0.2),length(bin_centers),length(bin_centers))
@time Fmatrix(F,[0.2, 0., 0.0],bin_centers) # Fi
F

  0.303338 seconds (141.05 k allocations: 5.048 MB, 3.61% gc time)


35x35 Array{Float64,2}:
 1.0  0.41182      0.218907     0.0917964    …  0.0          0.0          0.0
 0.0  0.197142     0.172131     0.112014        0.0          0.0          0.0
 0.0  0.187228     0.217923     0.187228        0.0          0.0          0.0
 0.0  0.119939     0.187228     0.217923        0.0          0.0          0.0
 0.0  0.0571377    0.119939     0.187228        0.0          0.0          0.0
 0.0  0.0202246    0.0571377    0.119939     …  0.0          0.0          0.0
 0.0  0.00531176   0.0202246    0.0571377       0.0          0.0          0.0
 0.0  0.00103206   0.00531176   0.0202246       0.0          0.0          0.0
 0.0  0.000148294  0.00103206   0.00531176      0.0          0.0          0.0
 0.0  1.59548e-5   0.000148294  0.00103206      0.0          0.0          0.0
 0.0  1.11105e-6   1.59548e-5   0.000148294  …  0.0          0.0          0.0
 0.0  0.0          1.11105e-6   1.59548e-5      0.0          0.0          0.0
 0.0  0.0          0.0          1.11105e

In [12]:
F = zeros(typeof(params[1]),length(bin_centers),length(bin_centers))
@time Fmatrix(F,params,bin_centers)
F

  0.001752 seconds (37.71 k allocations: 596.500 KB)


35x35 Array{Float64,2}:
 1.0  0.461125    0.365007    0.274946    …  0.0         0.0         0.0
 0.0  0.0908452   0.0859592   0.0811359      0.0         0.0         0.0
 0.0  0.0945948   0.100835    0.0946575      0.0         0.0         0.0
 0.0  0.0888839   0.0945533   0.100892       0.0         0.0         0.0
 0.0  0.0737024   0.0889555   0.0945117      0.0         0.0         0.0
 0.0  0.0612142   0.0736883   0.0890271   …  0.0         0.0         0.0
 0.0  0.044833    0.0612781   0.0736743      0.0         0.0         0.0
 0.0  0.0329193   0.0448354   0.0613421      0.0         0.0         0.0
 0.0  0.0212918   0.0329616   0.0448377      0.0         0.0         0.0
 0.0  0.0138235   0.021298    0.0330038      0.0         0.0         0.0
 0.0  0.0078944   0.0138445   0.0213041   …  0.0         0.0         0.0
 0.0  0.00453257  0.0078985   0.0138655      0.0         0.0         0.0
 0.0  0.00228512  0.00454054  0.00790261     3.13941e-7  0.0         0.0
 ⋮                         

## logProbRight 
### (params::Vector, RightClickTimes::Vector, LeftClickTimes::Vector, Nsteps::Int)

* params = [sigma_a, sigma_s, sigma_i, lambda, B, bias, phi, tau_phi, lapse]
* RightClickTimes vector with elements indicating times of right clicks
* LeftClickTimes vector with elements indicating times of left clicks
* Nsteps number of timesteps to simulate 

a (column vector representing distribution of values of accumulator a)

a_trace (length(bin_centers)-by-Nsteps+1), a trace of the distribution of a as 
    a function of time
    
c_trace (row vector Nsteps+1 long, effective value of c as 
    a function of time after adaptation)


In [13]:
"""
version with inter-click interval(ici) for c_eff_net / c_eff_tot (followed the matlab code) 
(which was using dt for c_eff)

function logProbRight(params::Vector, RightClickTimes::Vector, LeftClickTimes::Vector, Nsteps::Int)

    Nsteps            number of timesteps to simulate
    RightClickTimes   vector with elements indicating times of right clicks
    LeftClickTimes    vector with elements indicating times of left clicks

    a      (column vector representing distribution of values of accumulator a)

    a_trace (length(bin_centers)-by-Nsteps+1), a trace of the distribution of a as 
            a function of time
    c_trace (row vector Nsteps+1 long, effective value of c as 
            a function of time after adaptation)

Takes params (including biased params)
    sigma_a = params[1]; sigma_s_R = params[2]; sigma_s_L = params[3];
    sigma_i = params[4]; lambda = params[5]; B = params[6]; bias = params[7]; 
    phi = params[8]; tau_phi = params[9]; lapse_R = params[10]; lapse_L = params[11];
    input_gain_weight = params[12];

Returns the log of the probability that the agent chose Right. 
"""

function logProbRight(params::Vector, RightClickTimes::Array{Float64,1}, LeftClickTimes::Array{Float64,1}, Nsteps::Int)
#     sigma_a = params[1]; sigma_s = params[2]; sigma_i = params[3];
#     lambda = params[4]; B = params[5]; bias = params[6];
#     phi = params[7]; tau_phi = params[8]; lapse = params[9]
#     biased_sigma2_s = params[10]; biased_input = params[11]; biased_lapse = params[12];
    sigma_a = params[1]; sigma_s_R = params[2]; sigma_s_L = params[3];
    sigma_i = params[4]; lambda = params[5]; B = params[6]; bias = params[7];
    phi = params[8]; tau_phi = params[9]; lapse_R = params[10]; lapse_L = params[11];
    input_gain_weight = params[12];

    if isempty(RightClickTimes) RightClickTimes = zeros(0) end;
    if isempty(LeftClickTimes ) LeftClickTimes  = zeros(0) end;

    NClicks = zeros(Int, Nsteps);
    Lhere  = zeros(Int, length(LeftClickTimes));
    Rhere = zeros(Int, length(RightClickTimes));

    for i in 1:length(LeftClickTimes)
        Lhere[i] = ceil((LeftClickTimes[i]+epsilon)/dt)
    end
    for i in 1:length(RightClickTimes)
        Rhere[i] = ceil((RightClickTimes[i]+epsilon)/dt)
    end

    for i in Lhere
        NClicks[Int(i)] = NClicks[Int(i)]  + 1
    end
    for i in Rhere
        NClicks[Int(i)] = NClicks[Int(i)]  + 1
    end

    # === Upgrading from ForwardDiff v0.1 to v0.2
    # instead of using convert we can use floor(Int, ForwardDiff.Dual) and
    # ceil(Int, ForwardDiff.Dual)

    binN = ceil(Int, B/dx)#Int(ceil(my_B/dx))
    binBias = floor(Int, bias/dx) + binN+1
    binBias_hp = ceil(Int, bias/dx) + binN+1

    if binBias<1 binBias = 1; end
    if binBias>binN*2+1 binBias = binN*2+1; end

    if binBias_hp<1 binBias_hp = 1; end
    if binBias_hp>binN*2+1 binBias_hp = binN*2+1; end

    bin_centers = zeros(typeof(dx), binN*2+1)
    make_bins(bin_centers, B, dx, binN)

    # Visualization
    a_trace = zeros(length(bin_centers), Nsteps)
    c_trace = zeros(1, Nsteps)

    ## biased lapse rate (lapse_R, lapse_L)
    a0 = zeros(typeof(sigma_a),length(bin_centers))
    a0[1] = lapse_L/2;
    a0[end] = lapse_R/2;
    a0[binN+1] = 1-lapse_L/2-lapse_R/2;

    temp_l = [NumericPair(LeftClickTimes[i],-1) for i=1:length(LeftClickTimes)]
    temp_r = [NumericPair(RightClickTimes[i],1) for i=1:length(RightClickTimes)]
    allbups = sort!([temp_l; temp_r])

    if phi == 1
      c_eff = 1.
    else
      c_eff = 0.
    end
    cnt = 0

    Fi = zeros(typeof(sigma_i),length(bin_centers),length(bin_centers))
    Fmatrix(Fi,[sigma_i, 0., 0.0], bin_centers)
    a = Fi*a0;

    a_trace[:,1] = a;

    F0 = zeros(typeof(sigma_a),length(bin_centers),length(bin_centers))
    Fmatrix(F0,[sigma_a*dt, lambda, 0.0], bin_centers)
    for i in 2:Nsteps
        net_input = 0.
        c_eff_tot = 0.
        c_eff_net = 0.
        net_i_input = 0.
        net_c_input = 0.

        if NClicks[i-1]==0
            c_eff_tot = 0.
            c_eff_net = 0.
            a = F0*a
        else
            for j in 1:NClicks[i-1]
                if cnt != 0 || j != 1
                    ici = allbups[cnt+j].x - allbups[cnt+j-1].x

                    c_eff = 1 + (c_eff*phi - 1)*exp(-ici/tau_phi)
                    c_eff_tot = c_eff_tot + c_eff
                    c_eff_net = c_eff_net + c_eff*allbups[cnt+j].y

                    ## (input_gain_weight) 0 to 1 : 0-left, 1-right
                    net_c_input = (c_eff_tot+c_eff_net)/2 # right
                    net_i_input = c_eff_tot-net_c_input # left

                    # net_input = 2*input_gain_weight*net_c_input - 2*(1-input_gain_weight)*net_i_input
                    net_input = input_gain_weight*net_c_input - net_i_input
                elseif cnt==0 && j==1
                    ici = 0.
                    c_eff = 1 + (c_eff*phi - 1)*exp(-ici/tau_phi)

                    c_eff_tot = c_eff_tot + c_eff
                    c_eff_net = c_eff_net + c_eff*allbups[cnt+j].y

                    ## (input_gain_weight) 0 to 1 : 0-left, 1-right
                    net_c_input = (c_eff_tot+c_eff_net)/2 # right
                    net_i_input = c_eff_tot-net_c_input # left

                    # net_input = 2*input_gain_weight*net_c_input - 2*(1-input_gain_weight)*net_i_input
                    net_input = input_gain_weight*net_c_input - net_i_input
                end
                if j == NClicks[i-1]
                    cnt = cnt+j
                end
            end

            ## biased params added
            net_sigma = sigma_a*dt + (sigma_s_R*net_i_input)/total_rate + (sigma_s_L*net_c_input)/total_rate
            F = zeros(typeof(net_sigma),length(bin_centers),length(bin_centers))
            Fmatrix(F,[net_sigma, lambda, net_input/dt], bin_centers)
            a = F*a
            # println("step ",i, " : ", net_sigma," , ",c_eff_net," , ",c_eff_tot," , ",net_c_input," , ",net_i_input," , ",net_input)
            # println("step ",i, " : net_sigma = ", net_sigma," = ",sigma_a*dt," + ",sigma_s_L,"*",net_i_input,"/",total_rate," + ",(sigma_s_R*net_c_input)/total_rate)

        end
        # println("step ",i," : ",a)

        a_trace[:,i] = a
        c_trace[i] = c_eff*exp(lambda*dt)
    end


    # likelihood of poking right
    if binBias == binBias_hp
      pright = sum(a[binBias+1:end])+a[binBias]/2
    else
      pright = sum(a[binBias+2:end]) +
      a[binBias]*((bin_centers[binBias+1] - bias)/dx/2) +
      a[binBias+1]*(0.5 + (bin_centers[binBias+1] - bias)/dx/2)
    end
    if pright-1 < epsilon && pright > 1
        pright = 1
    end
    if pright < epsilon && pright > 0
        pright = 0
    end
    
    if pright < 0
        pright = 0
    end

    a_trace[binBias,:]=a_trace[binBias,:]*((bin_centers[binBias+1] - bias)/dx/2)
    a_trace[binBias+1,:]=a_trace[binBias,:]*(0.5 + (bin_centers[binBias+1] - bias)/dx/2)

#     # visualize
#     subplot(211)
#     plt = plot((0:(Nsteps-1))*dt,c_trace[:])
# #     print(maximum(a_trace,1))
# #     plt = plot((0:(Nsteps-1))*dt,maximum(a_trace,1)[:])

#     subplot(212)

#     imshow(a_trace, interpolation="none", cmap=ColorMap("hot"),
#     extent=[0,(Nsteps-1)*dt,bin_centers[1],bin_centers[end]], aspect="auto")
#     xticks((0:(Nsteps-1))*dt)
#     yticks(bin_centers)

    return log(pright)
end

function logLike(params::Vector, RightClickTimes::Array{Float64,1}, LeftClickTimes::Array{Float64,1}, Nsteps::Int, rat_choice::Int)
    # likelihood of poking the same way the data did
    if rat_choice > 0
        # println("Right")
        return logProbRight(params, RightClickTimes, LeftClickTimes, Nsteps)
    elseif rat_choice < 0
        # println("Left")
        return log(1 - exp(logProbRight(params, RightClickTimes, LeftClickTimes, Nsteps)))
    end
end

logLike (generic function with 1 method)

## single_trial
### (params::Vector, RightClickTimes::Vector, LeftClickTimes::Vector, Nsteps::Int, rat_choice::Int)

In [14]:
""" 
function (LL, LLgrad) = 
    single_trial(params::Vector, RightClickTimes::Vector, LeftClickTimes::Vector, Nsteps::Int, rat_choice::Int)

Computes the log likelihood according to Bing's model, and returns log likelihood, gradient

params is a vector whose elements, in order, are
    sigma_a    square root of accumulator variance per unit time sqrt(click units^2 per second)
    sigma_s    standard deviation introduced with each click (will get scaled by click adaptation)
    sigma_i    square root of initial accumulator variance sqrt(click units^2)
    lambda     1/accumulator time constant (sec^-1). Positive means unstable, neg means stable
    B          sticky bound height (click units)
    bias       where the decision boundary lies (click units)
    phi        click adaptation/facilitation multiplication parameter
    tau_phi    time constant for recovery from click adaptation (sec)
    lapse      2*lapse fraction of trials are decided randomly

rat_choice     should be either "R" or "L"


RETURNS:


"""
# === Upgrading from ForwardDiff v0.1 to v0.2
# for Retrieving Lower-Order Results
#     # old way
#     answer, results = ForwardDiff.hessian(f, x, AllResults)
#     v = ForwardDiff.value(results)
#     g = ForwardDiff.gradient(results)
#     h = ForwardDiff.hessian(results) # == answer

#     # new way
#     out = HessianResult(x)
#     ForwardDiff.hessian!(out, f, x)
#     v = ForwardDiff.value(out)
#     g = ForwardDiff.gradient(out)
#     h = ForwardDiff.hessian(out)

function single_trial(params::Vector, RightClickTimes::Array{Float64,1}, LeftClickTimes::Array{Float64,1}, Nsteps::Int, rat_choice::Int)
    function llikey(params::Vector)
        logLike(params, RightClickTimes, LeftClickTimes, Nsteps, rat_choice)
    end

    result =  DiffBase.GradientResult(params)
    
    ForwardDiff.gradient!(result, llikey, params);
    
    LL     = DiffBase.value(result)
    LLgrad = DiffBase.gradient(result)
   
    return LL, LLgrad
end

# function single_trial(params::Vector, RightClickTimes::Vector, LeftClickTimes::Vector, Nsteps::Int, rat_choice::Int)
#     function llikey(params::Vector)
#         logLike(params, RightClickTimes, LeftClickTimes, Nsteps, rat_choice)
#     end

#     result =  HessianResult(params) 
#     ForwardDiff.hessian!(result, llikey, params);
    
#     LL     = ForwardDiff.value(result)
#     LLgrad = ForwardDiff.gradient(result)
#     LLhessian = ForwardDiff.hessian(result)
   
#     return LL, LLgrad, LLhessian
# end

single_trial (generic function with 1 method)

In [15]:
### =============== testing 1 ================= ####

# Parameters
sigma_a = 1; sigma_s_R = 0.1; sigma_s_L = 0.1; sigma_i = 0.2; 
lam = -0.5; B = 10.; bias = .2; 
phi = 0.3; tau_phi = 0.1; lapse_R = 0.05*2; lapse_L = 0.1;
input_gain_weight = 0.5; 

params = [sigma_a, sigma_s_R, sigma_s_L, sigma_i, lam, B, bias, phi, tau_phi, lapse_R, lapse_L, input_gain_weight]   


RightClickTimes, LeftClickTimes, maxT, rat_choice = trialdata(ratdata["rawdata"], 1)
Nsteps = Int(cld(maxT,dt))

@time logLike(params, RightClickTimes, LeftClickTimes, Nsteps, rat_choice)
### =========================================== #### 

  2.609864 seconds (1.50 M allocations: 40.055 MB, 5.21% gc time)


-0.052233844862698206

In [16]:
### =============== testing 2 ================= ####
@time LL, LLgrad = single_trial(params, RightClickTimes, LeftClickTimes, Nsteps, rat_choice)
println(LL)
println(LLgrad)
### =========================================== ####

  4.074829 seconds (3.08 M allocations: 163.358 MB, 1.26% gc time)
-0.052233844862698206
[-0.002097286726694895,-0.00046040041743545217,-2.111552427493129e-5,-0.006998724210286463,-0.0005052759591115179,-0.0,0.005616932141251665,0.009095339919944863,-0.06443041056820667,-0.5262882750604839,0.0005227738508069007,-0.0004523516957455874]


In [ ]:
-0.05142945554068131
[-0.00042899336292765306,-6.933289173521957e-5,-5.956546833135365e-5,-0.0012615164543306363,-9.08726954078407e-5,-0.0,0.0007856615969410698,0.0030393293166007807,-0.013795529098181146,-0.5263118078823543,7.565023526742202e-5,-0.004958318007655772]

In [ ]:
LL

In [ ]:
LLgrad

# Maximize LL over parameter space
### Optimization with Optim.jl


In [17]:
function ComputeLL(params::Vector, ratdata, ntrials::Int)
    LL        = 0.
        
    for i in 1:ntrials
        RightClickTimes, LeftClickTimes, maxT, rat_choice = trialdata(ratdata, i)
        Nsteps = Int(ceil(maxT/dt))

        LLi = logLike(params, RightClickTimes, LeftClickTimes, Nsteps, rat_choice)
        LL        = LL + LLi;
    end
    
    LL = -LL
    return LL
end

ComputeLL (generic function with 1 method)

In [18]:
function ComputeGrad(params::Vector, ratdata, ntrials::Int)
    function WrapperLL(params::Vector)
        LL        = 0.

        for i in 1:ntrials
            RightClickTimes, LeftClickTimes, maxT, rat_choice = trialdata(ratdata, i)
            Nsteps = Int(ceil(maxT/dt))

            LLi = logLike(params, RightClickTimes, LeftClickTimes, Nsteps, rat_choice)
            LL        = LL + LLi;
        end

        LL = -LL
        return LL
    end

    result =  DiffBase.GradientResult(params)
    
    ForwardDiff.gradient!(result, WrapperLL, params);
    
    LL     = DiffBase.value(result)
    LLgrad = DiffBase.gradient(result)
   
    return LL, LLgrad
end

ComputeGrad (generic function with 1 method)

In [ ]:
@profile ComputeLL(params,ratdata["rawdata"],300)
Profile.print()
Profile.clear_malloc_data() 

In [ ]:
ComputeLL(params,ratdata["rawdata"],300)

In [ ]:
@time ComputeGrad(params,ratdata["rawdata"],300)

In [19]:
function ComputeHess(params::Vector, ratdata, ntrials::Int)
    function WrapperLL(params::Vector)
        LL        = 0.

        for i in 1:ntrials
            RightClickTimes, LeftClickTimes, maxT, rat_choice = trialdata(ratdata, i)
            Nsteps = Int(ceil(maxT/dt))

            LLi = logLike(params, RightClickTimes, LeftClickTimes, Nsteps, rat_choice)
            LL        = LL + LLi;
        end

        LL = -LL
        return LL
    end

    result =  DiffBase.HessianResult(params)
    
    ForwardDiff.hessian!(result, WrapperLL, params);
    
    LL     = DiffBase.value(result)
    LLgrad = DiffBase.gradient(result)
    LLHess = DiffBase.hessian(result)
   
    return LL, LLgrad, LLHess
end

ComputeHess (generic function with 1 method)

In [20]:
@time ComputeHess(params,ratdata["rawdata"],300)

LoadError: InterruptException:

In [21]:
function Likely_all_trials{T}(LL::AbstractArray{T,1},params::Vector, ratdata, ntrials::Int)     
    for i in 1:ntrials
        RightClickTimes, LeftClickTimes, maxT, rat_choice = trialdata(ratdata, i)
        Nsteps = Int(ceil(maxT/dt))

        LL[i] = logLike(params, RightClickTimes, LeftClickTimes, Nsteps, rat_choice)
    end
end

Likely_all_trials (generic function with 1 method)

In [22]:
ntrials = 27;
likely_all = zeros(ntrials)
@time Likely_all_trials(likely_all, params, ratdata["rawdata"], ntrials)
likely_all

  4.344104 seconds (30.61 M allocations: 485.620 MB, 5.37% gc time)


27-element Array{Float64,1}:
 -0.0522338
 -0.12174  
 -1.36315  
 -0.83066  
 -2.99333  
 -0.176191 
 -1.42718  
 -0.0833221
 -1.41135  
 -0.448566 
 -0.0723979
 -2.31043  
 -0.142859 
  ⋮        
 -2.59819  
 -0.0513919
 -0.0583206
 -0.0513265
 -0.0632553
 -0.0867048
 -0.380048 
 -0.064279 
 -0.146067 
 -0.451256 
 -0.0519001
 -0.0546648

In [23]:
# Parameters
sigma_a = 1; sigma_s_R = 0.1; sigma_s_L = 0.1; sigma_i = 0.2; 
lam = -0.0005; B = 10.; bias = .2; 
phi = 0.3; tau_phi = 0.1; lapse_R = 0.05*2; lapse_L = 0.1;
input_gain_weight = 0.1; 

params = [sigma_a, sigma_s_R, sigma_s_L, sigma_i, lam, B, bias, phi, tau_phi, lapse_R, lapse_L, input_gain_weight]   


ntrials = 10#3836

function LL_f(params::Vector)
    return ComputeLL(params, ratdata["rawdata"], ntrials)
end

function LL_g!(params::Vector, grads::Vector)
#     LL, LLgrad, LLhess = llikey(params)
    LL, LLgrad = ComputeGrad(params, ratdata["rawdata"], ntrials)
    for i=1:length(params)
        grads[i] = LLgrad[i]
    end
end

function LL_fg!(params::Vector, grads)
    LL, LLgrad = ComputeGrad(params, ratdata["rawdata"], ntrials)
    for i=1:length(params)
        grads[i] = LLgrad[i]
    end
    return LL
end

function LL_h!(params::Vector, hess)
    LL, LLgrad, LLhess = ComputeHess(params, ratdata["rawdata"], ntrials)
    for i=1:length(params)
        for j=1:length(params)
            hess[i,j] = LLhess[i,j]
        end
    end
end

d4 = DifferentiableFunction(LL_f,
                            LL_g!,
                            LL_fg!)

d = TwiceDifferentiableFunction(LL_f, LL_g!, LL_h!)

Optim.TwiceDifferentiableFunction(LL_f,LL_g!,fg!,LL_h!)

In [28]:
# Parameters
sigma_a = 1; sigma_s_R = 0.1; sigma_s_L = 0.1; sigma_i = 0.2; 
lam = -0.0005; B = 10.; bias = .2; 
phi = 0.3; tau_phi = 0.1; lapse_R = 0.05*2; lapse_L = 0.1;
input_gain_weight = 0.1; 

params = [sigma_a, sigma_s_R, sigma_s_L, sigma_i, lam, B, bias, phi, tau_phi, lapse_R, lapse_L, input_gain_weight]   
params = [2.63352, 0.287501, 0.201707, 0.501707, -0.21113, 6.23087, 1.487071, 0.537924, 0.69, 0.140861, 
    0.22412, 0.99205]


l = [0, 0, 0,        0, -5, 5,  -5, 0.01, 0.005, 0., 0., 0.]
u = [200, 200, 200, 30, +5, 25, +5, 1.2,    0.7, 1., 1., 1.]

res = optimize(d4, params, l, u, Fminbox(); 
         optimizer = GradientDescent, optimizer_o = Optim.Options(g_tol = 1e-12,
                                                                        x_tol = 1e-10,
                                                                        f_tol = 1e-64,                                                                        
                                                                        iterations = 200,
                                                                        store_trace = true,
                                                                        show_trace = true,
                                                                        extended_trace = true))


Iter     Function value   Gradient norm 
     0     2.421347e+00     5.202553e+00
 * Current step size: 1.0
 * g(x): [0.07518051239727674,0.0022452823732403973,0.0005671744879261336,0.7023829567059633,0.5155987469393554,-4.6401440608377656e-5,-1.9151636683182642,2.5071321538196503,-1.8637941175727333,5.202553008602679,-1.4594851692905653,1.5406553923116555]
 * x: [2.63352,0.287501,0.201707,0.501707,-0.21113,6.23087,1.487071,0.537924,0.69,0.140861,0.22412,0.99205]
     1     2.058936e+00     1.985241e+01
 * Current step size: 0.008643111009370498
 * g(x): [0.0855524726694587,0.0021213591146758332,0.0012910527333609416,0.6992275668439397,0.5075302438422641,-4.640142435945819e-5,-1.5013409346462467,2.177678326579748,19.852407579534614,5.092311093023381,-1.2816307938023757,1.402552663901155]
 * x: [2.632870212213265,0.287481608115014,0.20170210520144793,0.49563770036380644,-0.21558635529848605,6.230870401036554,1.5036238661587684,0.516262349584578,0.699997160339148,0.09603654991056157,0.23

Results of Optimization Algorithm
 * Algorithm: Fminbox with Gradient Descent
 * Starting Point: [2.63352,0.287501, ...]
 * Minimizer: [1.5338007977525594,0.7692989337156102, ...]
 * Minimum: 1.140474e-10
 * Iterations: 6
 * Convergence: true
   * |x - x'| < 1.0e-10: false
   * |f(x) - f(x')| / |f(x)| < 1.0e-64: true
   * |g(x)| < 1.0e-12: false
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 3555
 * Gradient Calls: 3555

In [ ]:
x_bf = res.f_minimum

In [ ]:
res

In [ ]:
res.minimum

In [ ]:
@time LL, LLgrad, LLhess = ComputeHess(params,ratdata,ntrials)

In [29]:
params = [1.5338007977525594,0.7692989337156102,0.7126524147595394,0.35869238391025315,-2.35791022247109,6.405552862313561,4.385627585294427,0.34767072943745175,0.5632538313112261,7.065515474629753e-19,0.5812087809501,0.005745530316158898]

results = Optim.optimize(d, params, NewtonTrustRegion(), Optim.Options(g_tol = 1e-64,
                                                                      iterations = 200,
                                                                      show_every = true,
                                                                      store_trace = true,
                                                                      extended_trace = true,
                                                                      show_trace = true))




Iter     Function value   Gradient norm 
     0     1.140474e-10     5.000000e-01
 * reached_subproblem_solution: true
 * g(x): [1.0712385603115008e-9,0.0,6.794394188543077e-11,5.274395847206317e-11,9.024368815806526e-10,0.0,-8.178415921938403e-10,6.76123442350948e-11,-9.103047444427705e-11,0.49999999997663996,-8.038349824793789e-11,1.0262279210971584e-9]
 * h(x): [9.043744997922408e-9 0.0 5.735873358282446e-10 4.45577613282274e-10 8.038450527205579e-9 0.0 -7.49799962152414e-9 5.745967395856812e-10 -7.732631941265243e-10 -2.1941586547194194e-10 -7.550360800449783e-10 9.160452255885405e-9
 0.0 0.0 0.0 0.0 0.0 0.0 -0.0 0.0 -0.0 0.0 -0.0 0.0
 5.735873358282445e-10 0.0 3.634395037670631e-11 2.826182385742226e-11 5.101604055982091e-10 0.0 -4.755627471916209e-10 1.204043833281651e-10 -1.6289650060571083e-10 -1.3916611053119791e-11 -4.788861178509305e-11 5.81259312833122e-10
 4.45577613282274e-10 0.0 2.826182385742225e-11 2.1423468322948495e-11 4.766946458570781e-10 0.0 -3.6916091802632515e-1

Results of Optimization Algorithm
 * Algorithm: Newton's Method (Trust Region)
 * Starting Point: [1.5338007977525594,0.7692989337156102, ...]
 * Minimizer: [1.5338007977525594,0.7692989337156102, ...]
 * Minimum: -0.000000e+00
 * Iterations: 16
 * Convergence: true
   * |x - x'| < 1.0e-32: false
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
   * |g(x)| < 1.0e-64: true
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 17
 * Gradient Calls: 17

In [ ]:
function main()
    
    ratname = "fof"
    # data import
    ratdata = matread(*("chrono_",ratname,"_rawdata.mat"))
    println("rawdata of ", ratname, " imported" )

    # number of trials
    ntrials = 2#length(ratdata["rawdata"]["rightbups"]) 

    # Parameters
    sigma_a = 1; sigma_s_R = 0.1; sigma_s_L = 0.1; sigma_i = 0.2; 
    lam = -0.0005; B = 10.; bias = .2; 
    phi = 0.3; tau_phi = 0.1; lapse_R = 0.05*2; lapse_L = 0.1;
    input_gain_weight = 0.1; 

    params = [sigma_a, sigma_s_R, sigma_s_L, sigma_i, lam, B, bias, phi, tau_phi, lapse_R, lapse_L, input_gain_weight]   

    l = [0, 0, 0,        0, -5, 5,  -5, 0.01, 0.005, 0., 0., 0.]
    u = [200, 200, 200, 30, +5, 25, +5, 1.2,    0.7, 1., 1., 1.]


    function LL_f(params::Vector)
        return ComputeLL(params, ratdata["rawdata"], ntrials)
    end

    function LL_g!(params::Vector, grads::Vector)
    #     LL, LLgrad, LLhess = llikey(params)
        LL, LLgrad = ComputeGrad(params, ratdata["rawdata"], ntrials)
        for i=1:length(params)
            grads[i] = LLgrad[i]
        end
    end

    function LL_fg!(params::Vector, grads)
        LL, LLgrad = ComputeGrad(params, ratdata["rawdata"], ntrials)
        for i=1:length(params)
            grads[i] = LLgrad[i]
        end
        return LL
    end
    
    d4 = DifferentiableFunction(LL_f,
                                LL_g!,
                                LL_fg!)

    tic()
    history = optimize(d4, params, l, u, Fminbox(); 
             optimizer = GradientDescent, optimizer_o = OptimizationOptions(g_tol = 1e-12,
                                                                            x_tol = 1e-10,
                                                                            f_tol = 1e-6,
                                                                            iterations = 200,
                                                                            store_trace = true,
                                                                            ))
    fit_time = toc()
    println(history.minimum)
    println(history)

    ## do a single functional evaluation at best fit parameters and save likely for each trial
    likely_all = zeros(typeof(sigma_i),ntrials)
    x_bf = history.minimum
    Likely_all_trials(likely_all, x_bf, ratdata["rawdata"], ntrials)
    LL, LLgrad, LLhess = ComputeHess(x_bf, ratdata, ntrials)

#     matwrite(saveto_filename, Dict([("ratname",ratname),
#                                     ("x_init",params),
#                                     ("trials",ntrials),
#                                     #("history",history),
#                                     ("f",history.f_minimum), 
#                                     ("x_converged",history.x_converged),
#                                     ("f_converged",history.f_converged),
#                                     ("g_converged",history.g_converged),                                    
#                                     ("fit_time",fit_time),
#                                     ("x_bf",history.minimum),
#                                     ("myfval", history.f_minimum),
#                                     ("hessian", LLhess),
#                                     ("likely",likely_all)
#                                     ]))

    
end

In [ ]:
main()

In [ ]:
# make grid of params. get likelihood with laplace approximation -> likelihood landscape

In [ ]:
## with hessian matrix
using PyPlot
imshow(log(abs(LLhess)), interpolation="none")

history = res
print(history.f_minimum)
X = history.minimum
tt = history.f_minimum * exp((-1/2)*transpose(X-history.minimum)*LLhess*(X-history.minimum))
print(tt)


## Simulation 
### generate synthetic data, test the model with several parameter sets
1. Generate Click trials
2. Run Virtual rats
3. Fit the synthetic data
4. compare the best-fit parameters with the parameters that we used to generate the data

In [ ]:
# function to make clicks.

function make_clicks_trains(difficulty, T, ntrials)

    # difficulty : 0.025 (39:1) / 0.075 (37:3) / 0.225 (31:9) / 0.35 (26:14)
    # T - max time (sec) : 0.5
    # ntrials : 1000

    R = 40

    # difficulty = r1:r2
    r1 = R*(1.0-difficulty);
    r2 = R-r1;

    # data = Array(Dict,ntrials)
    Ls = Array(Any,1,ntrials)
    Rs = Array(Any,1,ntrials)
    As = Array(Any,ntrials)
    Gams = Array(Any,ntrials)

    for t=1:ntrials

    if rand()<0.5
        gamma = log(r1/r2) # log of click rate ratio
    else
        gamma = -log(r1/r2)
    end
#     T = 0.5

    rrate = R/(exp(-gamma)+1)
    lrate = R - rrate

    srate = R*1000
    bup_width = 3
    lT2 = ceil(Int, T * 1000 / bup_width)
    tp1 = find(rand(lT2) .< lrate/(1000/bup_width))
    tp2 = find(rand(lT2) .< rrate/(1000/bup_width))

    prepend!(tp1,[0])
    prepend!(tp2,[0])

    leftbups = tp1 * (srate / (1000 / bup_width))/srate
    rightbups = tp2 * (srate / (1000 / bup_width))/srate

    answer = gamma > 0?1:-1

    Ls[t] = leftbups
    Rs[t] = rightbups
    As[t] = answer
    Gams[t] = gamma

    end

    data = Dict([("leftbups",Ls),
        ("gammas",Gams),
        ("rightbups",Rs),
        ("answer",As)])
    return data
end

In [ ]:
@time data = make_clicks_trains(0.075,0.8,10000)

In [ ]:
function clicks_from_data(data, ntrials)
    
    data_len = length(data["rightbups"]) 
    
    Ls = Array(Any,1,ntrials)
    Rs = Array(Any,1,ntrials)
    As = Array(Any,ntrials)
    Gams = Array(Any,ntrials)

    for t=1:ntrials
        i = mod(t, data_len)
        if i==0
            i = data_len;
        end
        Ls[t] = data["leftbups"][i]
        Rs[t] = data["rightbups"][i]
        As[t] = (data["hit"][i] == true) == (data["pokedR"][i] == true) ? 1 : -1
        Gams[t] = data["gamma"][i]
    end

    data = Dict([("leftbups",Ls),
        ("gamma",Gams),
        ("rightbups",Rs),
        ("answer",As)])
    return data
end

In [ ]:
data = clicks_from_data(ratdata["rawdata"], 1000)

In [ ]:
# run simulation : get virtual rat choice

# rats with different bias params.
# 1. strong post categorization (biased_lapse)
# 2. strong biased gain input (biased_input) 1~0 : >0.5 right bias, <0.5 left bias
# 3. accumulation shift (bias)
# 4. biased sensory noise (biased_noise)

# virtual rat
# # sigma_a / sigma_s_R / sigma_s_L / sigma_i / lambda / B / bias / phi / tau_phi / 
# # lapse_R / lapse_L / input_gain_weight

# params =[1.3  0.  0.   0. 0.06 10 0    0.33 0.39 0.03 0.5  0.5; # Strong Post Categorization
#          1.3  0.  0.   0. 0.06 10 0    0.33 0.39 0.03 0.03 0.1; # Strong biased gain input
#          1.3  0.  200. 0. 0.06 10 0    0.33 0.39 0.03 0.03 0.5; # Biased Sensory Noise
#          1.3  0.  0.   0. 0.06 10 -6.5 0.33 0.39 0.03 0.03 0.5; # Accumulation shift (bias)
#         ];

function run_virtual_rat(data, param, T)

    ntrials = length(data["leftbups"])
    Rs = Array(Any,1,ntrials)
    Ts = Array(Any,1,ntrials)
    Delta = Array(Int,1,ntrials)

    sigma_a = sqrt(param[1]); sigma_s_R = sqrt(param[2]); sigma_s_L = sqrt(param[3]);
    sigma_i = param[4]; lambda = param[5]; B = param[6]; bias = param[7]; 
    phi = param[8]; tau_phi = param[9]; lapse_R = param[10]; lapse_L = param[11];
    input_gain_weight = param[12];
    
    Flip = rand(ntrials) 
    Flip_RtoL = Flip .< lapse_L
    Flip_LtoR = Flip .> 1-lapse_R
    
    for i=1:ntrials
        leftbups = data["leftbups"][i]
        rightbups = data["rightbups"][i]
#         answer = data["answer"][i]

        rat_choice = 0;

        dt = 0.0001

        time = dt:dt:T
        lC = zeros(time)
        rC = zeros(time)
        dC = zeros(time)

        adap_c = zeros(time)
        adap_c[1] = 1.

        dd = exp(dt*lambda)
        eta = sigma_a*sqrt(dt)

#             figure()
        for t = 1:length(time)-1

            if isempty(find(time[t] .< leftbups .<= time[t+1])) & isempty(find(time[t] .< rightbups .<= time[t+1]))
                adap_c[t+1] = adap_c[t] + ((1-adap_c[t])/tau_phi + (phi-1)*adap_c[t])*dt
            else
                adap_c[t+1] = adap_c[t] + (1-adap_c[t])/tau_phi*dt
            end

            # sigma_i ?
#             if t==1
#                 lC[1] = sigma_i
#                 rc[1] = sigma_i
#             end
            
            if ~isempty(find(time[t] .< leftbups .<= time[t+1]))
                lC[t+1] = lC[t] + adap_c[t+1] + (randn()*sigma_s_L)
            else
                lC[t+1] = lC[t]*dd + randn()*eta

            end

            if ~isempty(find(time[t] .< rightbups .<= time[t+1]))
                rC[t+1] = rC[t] + adap_c[t+1] + (randn()*sigma_s_R)
            else
                rC[t+1] = rC[t]*dd + randn()*eta
            end

            lC[t+1] = max(0,lC[t+1])
            rC[t+1] = max(0,rC[t+1])
            
            dC[t+1] = (2*input_gain_weight*rC[t+1] - 2*(1-input_gain_weight)*lC[t+1]) + bias #rC[t+1] - lC[t+1]

            if dC[t+1] >= B
                dC[t+1:end] = dC[t+1]
                rat_choice = 1;
                break;
            elseif dC[t+1] <= -B
                dC[t+1:end] = dC[t+1]
                rat_choice = 0;
                break;
            end
        end

#             plot(time,dC,color="black")
#             plot(reshape(leftbups,(1,size(leftbups,1))),-B+0.25,color="blue",linestyle="none",marker="^",markeredgecolor="none")
#             plot(reshape(rightbups,(1,size(rightbups,1))),B-0.25,color="red",linestyle="none",marker="v",markeredgecolor="none")

#             new_position = [0.06,0.06,0.91,0.27] # Position Method 2
#             ax = gca()
#             ax[:set_position](new_position)
#             yticks(-B:1:B)
#             grid()

        if rat_choice == 0
            rat_choice = dC[end] > 0?1:0
        end

        if Flip_RtoL[i]
            rat_choice = rat_choice == 1?0:0
        elseif Flip_LtoR[i]
            rat_choice = rat_choice == 0?1:1
        end
        
        Rs[i] = rat_choice
        if size(leftbups,1) > 1 || size(rightbups,1) > 1
            Ts[i] = maximum([leftbups; rightbups]) + 0.1
        else
            Ts[i] = maximum([leftbups rightbups]) + 0.1
        end
        Delta[i] = length(rightbups) - length(leftbups);

    end
    results =  Dict([("leftbups", data["leftbups"]),
        ("rightbups", data["rightbups"]),
#         ("answer", data["answer"]),
        ("pokedR", Rs),
        ("params", param),
        ("Delta", Delta),
        ("T",Ts)
        ])
    return results
end



In [ ]:
# sigma_a / sigma_s_R / sigma_s_L / sigma_i / lambda / B / bias / phi / tau_phi / 
# lapse_R / lapse_L / input_gain_weight

params =[1.3  0.  0.   0. 0.06 10 0    0.33 0.39 0.03 0.5  0.5; # Strong Post Categorization
         1.3  0.  0.   0. 0.06 10 0    0.33 0.39 0.03 0.03 0.1; # Strong biased gain input
         1.3  0.  200. 0. 0.06 10 0    0.33 0.39 0.03 0.03 0.5; # Biased Sensory Noise
         1.3  0.  0.   0. 0.06 10 -6.5 0.33 0.39 0.03 0.03 0.5; # Accumulation shift (bias)
        ];
ntrials = 10000



In [ ]:
## run virtual rat with 4 different parameter set.
tic()
results1 = run_virtual_rat(data,params[1,:],0.5)
C1 = sort(unique(results1["Delta"]))
prob_right1 = zeros(length(C1))
for i=1:length(C1)
    ci = C1[i]
    idd = find(ci.==results1["Delta"])
    prob_right1[i] = sum(results1["pokedR"][idd])/length(idd)
end
println("set1 done")
println(toc())

results2 = run_virtual_rat(data,params[2,:],0.5)
C2 = sort(unique(results2["Delta"]))
prob_right2 = zeros(length(C2))
for i=1:length(C2)
    ci = C2[i]
    idd = find(ci.==results2["Delta"])
    prob_right2[i] = sum(results2["pokedR"][idd])/length(idd)
end
println("set2 done")

results3 = run_virtual_rat(data,params[3,:],0.5)
C3 = sort(unique(results3["Delta"]))
prob_right3 = zeros(length(C3))
for i=1:length(C3)
    ci = C3[i]
    idd = find(ci.==results3["Delta"])
    prob_right3[i] = sum(results3["pokedR"][idd])/length(idd)
end
println("set3 done")

results4 = run_virtual_rat(data,params[4,:],0.5)
C4 = sort(unique(results4["Delta"]))
prob_right4 = zeros(length(C4))
for i=1:length(C4)
    ci = C4[i]
    idd = find(ci.==results4["Delta"])
    prob_right4[i] = sum(results4["pokedR"][idd])/length(idd)
end
println("set4 done")



In [ ]:
figure; 
subplot(221); plot(C1,prob_right1,linestyle="none",marker=".")
subplot(222); plot(C2,prob_right2,linestyle="none",marker=".")
subplot(223); plot(C3,prob_right3,linestyle="none",marker=".")
subplot(224); plot(C4,prob_right4,linestyle="none",marker=".")



In [ ]:
using LsqFit

In [ ]:
sig4(x, p) = p[1] + p[2]./(1 + exp(-(x-p[3]) ./ p[4]) )

In [ ]:
figure; 
subplot(221); plot(C1,prob_right1,linestyle="none",marker=".")
fit = curve_fit(sig4, C1, prob_right1, [0.1, 0.2, 0.5, 0.2])
x = minimum(C1):0.1:maximum(C1)
y = sig4(x,fit.param)
plot(x,y)
ylim(0,1)
# xlabel("#(right) - #(left)")
ylabel("prob. of right poke")
title("Strong Post Categorization")
println("1: lapse : ", fit.param[1], ", bias  : ", fit.param[3])

subplot(222); plot(C2,prob_right2,linestyle="none",marker=".")
fit = curve_fit(sig4, C2, prob_right2, [0.1, 0.2, 0.5, 0.2])
x = minimum(C2):0.1:maximum(C2)
y = sig4(x,fit.param)
plot(x,y)
ylim(0,1)
# xlabel("#(right) - #(left)")
ylabel("prob. of right poke")
title("Strong biased gain input")
println("2: lapse : ", fit.param[1], ", bias  : ", fit.param[3])

subplot(223); plot(C3,prob_right3,linestyle="none",marker=".")
fit = curve_fit(sig4, C3, prob_right3, [0.1, 0.2, 0.5, 0.2])
x = minimum(C3):0.1:maximum(C3)
y = sig4(x,fit.param)
plot(x,y)
ylim(0,1)
xlabel("#(right) - #(left)")
ylabel("prob. of right poke")
title("Biased Sensory Noise")
println("3: lapse : ", fit.param[1], ", bias  : ", fit.param[3])

subplot(224); plot(C4,prob_right4,linestyle="none",marker=".")
fit = curve_fit(sig4, C4, prob_right4, [0.1, 0.2, 0.5, 0.2])
x = minimum(C4):0.1:maximum(C4)
y = sig4(x,fit.param)
plot(x,y)
ylim(0,1)
xlabel("#(right) - #(left)")
ylabel("prob. of right poke")
title("Accumulation shift (bias)")
println("4: lapse : ", fit.param[1], ", bias  : ", fit.param[3])



In [ ]:
p = 2;
println("set",p, " done")

In [ ]:
*("set",string(p), " done")

In [ ]:
figure; subplot(5,2,p)